In [2]:
import pandas as pd
import numpy as np

In [11]:
X = np.matrix([[1,2,2],[2,1,1],[2,2,1]])
X.transpose().dot(X)

matrix([[9, 8, 6],
        [8, 9, 7],
        [6, 7, 6]])

In [13]:
class OlsRegressor:
    # Class Variable
    name = 'Mutiple Linear Regression'
    
    # The init method or constructor
    def __init__(self, X,Y,solver):
        self.X = X
        self.Y = Y
        self.solver = solver
        self.coef = (X.transpose()*X)**(-1)*(X.transpose()*Y)
        
    def predict(self,X_test):
        return X_test*self.coef

In [39]:
X = np.matrix([[1,2],[1,1],[1,2],[1,3],[1,5],[1,2],[1,4],[1,10],[1,3]])
Y = X[:,1]*2+np.matrix(np.random.randn(len(X))).transpose()

In [42]:
lm = OlsRegressor(X,Y,'OLS')

In [44]:
lm.coef

matrix([[-0.35464123],
        [ 1.9520034 ]])

In [45]:
X_test = X*2

In [46]:
lm.predict(X_test)

matrix([[ 3.54936556],
        [ 1.59736217],
        [ 3.54936556],
        [ 5.50136896],
        [ 9.40537575],
        [ 3.54936556],
        [ 7.45337236],
        [19.16539274],
        [ 5.50136896]])

In [63]:
class LogisticRegression(LinearClassifierMixin,SparseCoefMixin,BaseEstimator):
        def __init__(self, penalty='l2', *, dual=False, tol=1e-4, C=1.0,
                 fit_intercept=True, intercept_scaling=1, class_weight=None,
                 random_state=None, solver='lbfgs', max_iter=100,
                 multi_class='auto', verbose=0, warm_start=False, n_jobs=None,
                 l1_ratio=None):
            self.penalty = penalty
            self.dual = dual
            self.tol = tol
            self.C = C
            self.fit_intercept = fit_intercept
            self.intercept_scaling = intercept_scaling
            self.class_weight = class_weight
            self.random_state = random_state
            self.solver = solver
            self.max_iter = max_iter
            self.multi_class = multi_class
            self.verbose = verbose
            self.warm_start = warm_start
            self.n_jobs = n_jobs
            self.l1_ratio = l1_ratio
            
        def fit(self, X, y, sample_weight=None):    
            solver = _check_solver(self.solver, self.penalty, self.dual)
            ## 报错信息
            if not isinstance(self.C, numbers.Number) or self.C < 0:
                raise ValueError("Penalty term must be positive; got (C=%r)"
                                 % self.C)
            if self.penalty == 'elasticnet':
                if (not isinstance(self.l1_ratio, numbers.Number) or
                        self.l1_ratio < 0 or self.l1_ratio > 1):
                    raise ValueError("l1_ratio must be between 0 and 1;"
                                     " got (l1_ratio=%r)" % self.l1_ratio)
            elif self.l1_ratio is not None:
                warnings.warn("l1_ratio parameter is only used when penalty is "
                              "'elasticnet'. Got "
                              "(penalty={})".format(self.penalty))
            if self.penalty == 'none':
                if self.C != 1.0:  # default values
                    warnings.warn(
                        "Setting penalty='none' will ignore the C and l1_ratio "
                        "parameters"
                    )
                    # Note that check for l1_ratio is done right above
                C_ = np.inf
                penalty = 'l2'
            else:
                C_ = self.C
                penalty = self.penalty
            if not isinstance(self.max_iter, numbers.Number) or self.max_iter < 0:
                raise ValueError("Maximum number of iteration must be positive;"
                                 " got (max_iter=%r)" % self.max_iter)
            if not isinstance(self.tol, numbers.Number) or self.tol < 0:
                raise ValueError("Tolerance for stopping criteria must be "
                                 "positive; got (tol=%r)" % self.tol)

            if solver == 'lbfgs':
                _dtype = np.float64
            else:
                _dtype = [np.float64, np.float32]
        
        # -------------------------------------------------------------------------------------------
        X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
                                   order="C",
                                   accept_large_sparse=solver != 'liblinear')
        check_classification_targets(y)
        self.classes_ = np.unique(y)
        multi_class = _check_multi_class(self.multi_class, solver,
                                         len(self.classes_))
#                 if solver == 'liblinear':
#                     if effective_n_jobs(self.n_jobs) != 1:
#                         warnings.warn("'n_jobs' > 1 does not have any effect when"
#                                       " 'solver' is set to 'liblinear'. Got 'n_jobs'"
#                                       " = {}.".format(effective_n_jobs(self.n_jobs)))
#                     self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
#                         X, y, self.C, self.fit_intercept, self.intercept_scaling,
#                         self.class_weight, self.penalty, self.dual, self.verbose,
#                         self.max_iter, self.tol, self.random_state,
#                         sample_weight=sample_weight)
#                     self.n_iter_ = np.array([n_iter_])
#                     return self

        if solver in ['sag', 'saga']:
            max_squared_sum = row_norms(X, squared=True).max()
        else:
            max_squared_sum = None

        n_classes = len(self.classes_)
        classes_ = self.classes_
        if n_classes < 2:
            raise ValueError("This solver needs samples of at least 2 classes"
                             " in the data, but the data contains only one"
                             " class: %r" % classes_[0])

        if len(self.classes_) == 2:
            n_classes = 1
            classes_ = classes_[1:]

        if self.warm_start:
            warm_start_coef = getattr(self, 'coef_', None)
        else:
            warm_start_coef = None
        if warm_start_coef is not None and self.fit_intercept:
            warm_start_coef = np.append(warm_start_coef,
                                        self.intercept_[:, np.newaxis],
                                        axis=1)

        # Hack so that we iterate only once for the multinomial case.
        if multi_class == 'multinomial':
            classes_ = [None]
            warm_start_coef = [warm_start_coef]
        if warm_start_coef is None:
            warm_start_coef = [None] * n_classes

        path_func = delayed(_logistic_regression_path)

        # The SAG solver releases the GIL so it's more efficient to use
        # threads for this solver.
        if solver in ['sag', 'saga']:
            prefer = 'threads'
        else:
            prefer = 'processes'
        fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
                               **_joblib_parallel_args(prefer=prefer))(
            path_func(X, y, pos_class=class_, Cs=[C_],
                      l1_ratio=self.l1_ratio, fit_intercept=self.fit_intercept,
                      tol=self.tol, verbose=self.verbose, solver=solver,
                      multi_class=multi_class, max_iter=self.max_iter,
                      class_weight=self.class_weight, check_input=False,
                      random_state=self.random_state, coef=warm_start_coef_,
                      penalty=penalty, max_squared_sum=max_squared_sum,
                      sample_weight=sample_weight)
            for class_, warm_start_coef_ in zip(classes_, warm_start_coef))

        fold_coefs_, _, n_iter_ = zip(*fold_coefs_)
        self.n_iter_ = np.asarray(n_iter_, dtype=np.int32)[:, 0]

        n_features = X.shape[1]
        if multi_class == 'multinomial':
            self.coef_ = fold_coefs_[0][0]
        else:
            self.coef_ = np.asarray(fold_coefs_)
            self.coef_ = self.coef_.reshape(n_classes, n_features +
                                            int(self.fit_intercept))

        if self.fit_intercept:
            self.intercept_ = self.coef_[:, -1]
            self.coef_ = self.coef_[:, :-1]
        else:
            self.intercept_ = np.zeros(n_classes)
        return self
    
        def predict_proba(self, X):
            check_is_fitted(self)

        ovr = (self.multi_class in ["ovr", "warn"] or
               (self.multi_class == 'auto' and (self.classes_.size <= 2 or
                                                self.solver == 'liblinear')))
        if ovr:
            return super()._predict_proba_lr(X)
        else:
            decision = self.decision_function(X)
            if decision.ndim == 1:
                # Workaround for multi_class="multinomial" and binary outcomes
                # which requires softmax prediction with only a 1D decision.
                decision_2d = np.c_[-decision, decision]
            else:
                decision_2d = decision
            return softmax(decision_2d, copy=False)
            
            

SyntaxError: 'return' outside function (Temp/ipykernel_23536/2908430064.py, line 151)

In [67]:
class LinearRegression(MultiOutputMixin, RegressorMixin, LinearModel):
    @_deprecate_positional_args
    def __init__(self, *, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=None, positive=False):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs
        self.positive = positive

    def fit(self, X, y, sample_weight=None):

        n_jobs_ = self.n_jobs

        accept_sparse = False if self.positive else ['csr', 'csc', 'coo']

        X, y = self._validate_data(X, y, accept_sparse=accept_sparse,
                                   y_numeric=True, multi_output=True)

        if sample_weight is not None:
            sample_weight = _check_sample_weight(sample_weight, X,
                                                 dtype=X.dtype)

        X, y, X_offset, y_offset, X_scale = self._preprocess_data(
            X, y, fit_intercept=self.fit_intercept, normalize=self.normalize,
            copy=self.copy_X, sample_weight=sample_weight,
            return_mean=True)

        if sample_weight is not None:
            # Sample weight can be implemented via a simple rescaling.
            X, y = _rescale_data(X, y, sample_weight)

        if self.positive:
            if y.ndim < 2:
                self.coef_, self._residues = optimize.nnls(X, y)
            else:
                # scipy.optimize.nnls cannot handle y with shape (M, K)
                outs = Parallel(n_jobs=n_jobs_)(
                        delayed(optimize.nnls)(X, y[:, j])
                        for j in range(y.shape[1]))
                self.coef_, self._residues = map(np.vstack, zip(*outs))
        elif sp.issparse(X):
            X_offset_scale = X_offset / X_scale

            def matvec(b):
                return X.dot(b) - b.dot(X_offset_scale)

            def rmatvec(b):
                return X.T.dot(b) - X_offset_scale * np.sum(b)

            X_centered = sparse.linalg.LinearOperator(shape=X.shape,
                                                      matvec=matvec,
                                                      rmatvec=rmatvec)

            if y.ndim < 2:
                out = sparse_lsqr(X_centered, y)
                self.coef_ = out[0]
                self._residues = out[3]
            else:
                # sparse_lstsq cannot handle y with shape (M, K)
                outs = Parallel(n_jobs=n_jobs_)(
                    delayed(sparse_lsqr)(X_centered, y[:, j].ravel())
                    for j in range(y.shape[1]))
                self.coef_ = np.vstack([out[0] for out in outs])
                self._residues = np.vstack([out[3] for out in outs])
        else:
            self.coef_, self._residues, self.rank_, self.singular_ = \
                linalg.lstsq(X, y)
            self.coef_ = self.coef_.T

        if y.ndim == 1:
            self.coef_ = np.ravel(self.coef_)
        self._set_intercept(X_offset, y_offset, X_scale)
        return self    

NameError: name '_deprecate_positional_args' is not defined

In [65]:
import sklearn

In [ ]:
def Classifier_CV(Xdata,Ydata,class_model,size_pct=0.2,model="Temp",rn=0,i=1):
    globals()['Model_{}_{}'.format(model,i)] = class_model()
    globals()['X_train_{}_{}'.format(model,i)],globals()['X_test_{}_{}'.format(model,i)],globals()['Y_train_{}_{}'.format(model,i)],globals()['Y_test_{}_{}'.format(model,i)] = train_test_split(Xdata, Ydata, test_size=size_pct, random_state=rn)
    globals()['X_trainabs_{}_{}'.format(model,i)],globals()['X_valid_{}_{}'.format(model,i)],globals()['Y_trainabs_{}_{}'.format(model,i)],globals()['Y_valid_{}_{}'.format(model,i)] =train_test_split(globals()['X_train_{}_{}'.format(model,i)],globals()['Y_train_{}_{}'.format(model,i)],test_size=size_pct,random_state=rn)
    globals()['Model_{}_{}'.format(model,i)].fit(globals()['X_train_{}_{}'.format(model,i)],globals()['Y_train_{}_{}'.format(model,i)])
    #在train上预测
    globals()['Y_train_predict_CV_{}_{}'.format(model,i)] = cross_val_predict(globals()['Model_{}_{}'.format(model,i)],globals()['X_train_{}_{}'.format(model,i)],globals()['Y_train_{}_{}'.format(model,i)],cv=5)
    # 在test集合上预测---------------------------------------------------------------------------------
    globals()['Y_test_predict_CV_{}_{}'.format(model,i)] = cross_val_predict(globals()['Model_{}_{}'.format(model,i)],globals()['X_test_{}_{}'.format(model,i)],globals()['Y_test_{}_{}'.format(model,i)],cv=5)
    # 得到AUC ---------------------------------------------------------------------------------------
    globals()['Y_test_prob_CV_{}_{}'.format(model,i)] = cross_val_predict(globals()['Model_{}_{}'.format(model,i)],globals()['X_test_{}_{}'.format(model,i)],globals()['Y_test_{}_{}'.format(model,i)],cv=5,method='predict_proba')
    globals()['fpr_CV_{}_{}'.format(model,i)], globals()['tpr_CV_{}_{}'.format(model,i)], globals()['thresholds_CV_{}_{}'.format(model,i)] = roc_curve(globals()['Y_test_{}_{}'.format(model,i)], (globals()['Y_test_prob_CV_{}_{}'.format(model,i)])[:, 1])
    globals()['auc_CV_{}_{}'.format(model,i)] = auc(globals()['fpr_CV_{}_{}'.format(model,i)], globals()['tpr_CV_{}_{}'.format(model,i)])
    return globals()['auc_CV_{}_{}'.format(model,i)]

In [ ]:
print("Confusion matrix (training):\n {0}\n".format(confusion_matrix(globals()['{}_Y_train_{}'.format(model,i)], globals()['{}_Y_train_predict_{}'.format(model,i)])))
print("Classification report (training):\n {0}".format(classification_report(globals()['{}_Y_train_{}'.format(model,i)], globals()['{}_Y_train_predict_{}'.format(model,i)])))